# Libraries

In [1]:
import sys
sys.path.append('..')

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import pandas as pd
import graph_tool as gt
from graph_tool.all import vertex_percolation
from tqdm import tqdm
from operator import itemgetter
import itertools
import os

import MuxVizPy as mxp

In [6]:
working_dir = "data"

In [ ]:
human_nodes = pd.read_csv("data/BIOSTR_homo_sapiens.nodes", sep=" ")
human_map = dict(zip(human_nodes['nodeSymbol'], np.arange(len(human_nodes))))

virus_names = np.sort(os.listdir("data/SyntheticViruses/original"))
virus_metadata = pd.read_csv("data/viruses_metadata.csv", header=0, sep=";")
virus_onco = virus_metadata[virus_metadata["isOncogenic"] == True].virus.unique()
virus_nonco = virus_metadata[virus_metadata["isOncogenic"] == False].virus.unique()

synt_files = np.sort(os.listdir(working_dir+"/SyntheticViruses/"))[:5]

virus_dict = {i : vname for i, vname in enumerate(virus_names)}

virus_onco_idx = np.where(np.isin(virus_names, virus_onco))[0]
virus_nonco_idx = np.where(np.isin(virus_names, virus_nonco))[0]

# Index Lists

Pick random samples of oncogenic and non-oncogenic viruses in different proportion to generate the combination sets. 

In [4]:
n_iters = 256
np.random.seed(100)

#N
n_virus_indexes = np.array([np.random.choice(virus_nonco_idx, 4, replace=False) for i in range(n_iters)])

#N1O
n1o_virus_indexes = []
for i in range(n_iters):
    onco_pick = np.random.choice(virus_onco_idx, 1, replace=False)
    nonco_pick = np.random.choice(virus_nonco_idx, 3, replace=False)
    n1o_virus_indexes.append(np.concatenate([nonco_pick, onco_pick]))
n10_virus_indexes = np.array(n1o_virus_indexes)

#N2O
n2o_virus_indexes = []
for i in range(n_iters):
    onco_pick = np.random.choice(virus_onco_idx, 2, replace=False)
    nonco_pick = np.random.choice(virus_nonco_idx, 2, replace=False)
    n2o_virus_indexes.append(np.concatenate([nonco_pick, onco_pick]))
n20_virus_indexes = np.array(n2o_virus_indexes)

#N3O
n3o_virus_indexes = []
for i in range(n_iters):
    onco_pick = np.random.choice(virus_onco_idx, 3, replace=False)
    nonco_pick = np.random.choice(virus_nonco_idx, 1, replace=False)
    n3o_virus_indexes.append(np.concatenate([nonco_pick, onco_pick]))
n3o_virus_indexes = np.array(n3o_virus_indexes)

#O
comb = list(itertools.combinations(range(8), 4))
o_virus_indexes = np.array([list(virus_onco_idx[list(comb[i])]) for i in range(len(comb))])

In [5]:
#organize in lists to better handle later

index_lists = [n_virus_indexes, n1o_virus_indexes, n2o_virus_indexes, n3o_virus_indexes, o_virus_indexes]
names_lists=["n", "n1o", "n2o", "n3o", "o"]

for i in range(len(index_lists)):
    print(f"Length of {names_lists[i]}: {len(index_lists[i])}")

Length of n: 256
Length of n1o: 256
Length of n2o: 256
Length of n3o: 256
Length of o: 70


In [5]:
#save the indexes combination for reproducibility    
for i in range(len(index_lists)):
    np.savetxt(X=index_lists[i], fname="data/MultilayerIndexes/"+names_lists[i]+".txt", fmt="%d")

# Topological quantities extraction

## LVC

The analysis of the Largest Intersected Components (LICs) allows us to investigate common features among viruses within the same multilayer network, particularly by identifying a shared core of nodes involved in the PPIs of viruses in the same category. Additionally, the LIC can be compared with the corresponding **Largest Viable Component (LVC)**, which is a subset of the LIC and represents the set of nodes that are simultaneously connected by the same path in all layers.

<img src="images/LVC.png" alt="lvc" width="300"/>

In [ ]:
### LVC computation for each set of combinations ###
####################################################

if not os.path.isdir(working_dir+"/LVC/original"):
    os.mkdir(working_dir+"/LVC/original")
for nam, lst in zip(names_lists, index_lists):
    print("Computing LVC for set:", nam)
    lvc_size=[]
    for i in tqdm(range(len(lst))):
        net = mxp.VirusMultiplex_from_dirlist([working_dir+"/SyntheticViruses/original/"+a for a in itemgetter(*lst[i])(virus_dict)])
        lvc_curr = mxp.topology.get_multi_LVC(net.g_list, printt=False)
        if type(lvc_curr)==int:
            lvc_curr = [lvc_curr]
        lvc_size.append(len(lvc_curr))

    np.savetxt(X=lvc_size, fname=working_dir+"/LVC/original/"+nam+".txt", fmt="%d")

In [ ]:

#for each realization of the randomization of the human PPI network
for s in synt_files:
    print("Processing synthetic set:", s)
    if not os.path.isdir(working_dir+"/LVC/"+str(s)):
        os.mkdir(working_dir+"/LVC/"+str(s))
    for nam, lst in zip(names_lists, index_lists):
        print("Computing LVC for set:", nam)
        lvc_size=[]
        for i in tqdm(range(len(lst))):
            net = mxp.VirusMultiplex_from_dirlist([working_dir+"/SyntheticViruses/"+str(s)+"/"+a for a in itemgetter(*lst[i])(virus_dict)])
            
            lvc_curr = mxp.topology.get_multi_LVC(net.g_list, printt=False)
    
            if type(lvc_curr)==int:
                lvc_curr = [lvc_curr]
            lvc_size.append(len(lvc_curr))
    
        np.savetxt(X=lvc_size, fname=working_dir+"/LVC/"+str(s)+"/"+nam+"_lvc.txt", fmt="%d")
        

Processing synthetic set: 0
Computing LVC for set: n


100%|██████████| 256/256 [03:22<00:00,  1.26it/s]


Computing LVC for set: n1o


 98%|█████████▊| 250/256 [06:13<00:08,  1.41s/it]

In [ ]:
# getting the list of nodes in the LVCs
# the resulting file will be used for the GO enrichment analysis

# original network
net = mxp.VirusMultiplex_from_dirlist([working_dir+"/SynteticViruses/original/"+a for a in virus_metadata.iloc[onco_virus_indexes]["virus"]])  
lvc_curr = mxp.topology.get_multi_LVC(net.g_list, printt=False)
np.savetxt(X=np.array(list(net.node_map))[lvc_curr], fname=working_dir+"/GOdata/genes.list", fmt="%s")

# saving union of nodes in onco layers of the original net
# to be used as background gene set for the GO enrichment analysis
#np.savetxt(X=list(net.node_map.keys()), fname=working_dir+"/GOdata/gobackground.list", fmt="%s")

#randomized networks
for k in tqdm(range(500)):
    net = mxp.VirusMultiplex_from_dirlist([working_dir+"/SynteticViruses/"+str(k)+"/"+a for a in virus_metadata.iloc[onco_virus_indexes]["virus"]])
            
    lvc_curr = mxp.topology.get_multi_LVC(net.g_list, printt=False)
    lvc_synt=np.array(list(net.node_map))[lvc_curr]
    np.savetxt(X=lvc_synt, fname=working_dir+"/GOdata/Synt/genes_"+str(k)+".list", fmt="%s")

## Percolation

Another approach to analyze the properties of the systems is to examine their **robustness to percolation processes**. Percolation, specifically nodes percolation, involves progressively removing nodes from the networks. The order in which nodes are removed, along with their corresponding links, in the multilayer networks is determined by their descending order of pagerank versatility when considering the system as an edge-colored network.
In this analysis, the focus is on the critical point, which refers to the fraction of removed nodes at which a peak in the dimension of the second largest component is observed. This critical point provides insights into the network’s resistance to targeted attacks, which, from a biological perspective, could correspond to the action of certain drugs.

<img src="images/criticPoint.png" alt="percolation" width="400"/>

In [76]:
### Node percolation (page-rank) analysis ###
#############################################

if not os.path.isdir(working_dir+"/percolation/original"):
    os.mkdir(working_dir+"/percolation/original")
for nam, lst in zip(names_lists, index_lists):
    print("Computing percolation points for set:", nam)
    perc_list=[]
    for i in tqdm(range(len(lst))):
        # create multiplex network for the current sample
        net = mxp.VirusMultiplex_from_dirlist([working_dir+"/SyntheticViruses/original/"+a for a in itemgetter(*lst[i])(virus_dict)])
        tensor=mxp.build.get_node_tensor_from_network_list(net.g_list)

        # define node removal order based on pagerank versatility
        order = mxp.versatility.get_multi_RW_centrality_edge_colored(tensor)
        order = order.sort_values("vers")["phy nodes"].to_numpy()
        g_agg = mxp.build.get_aggregate_network(tensor)

        # compute critical point based on second largest component size
        perc_agg_2 = vertex_percolation(g_agg, order, second=True)[0]
        max_perc = np.argmax(perc_agg_2)/len(perc_agg_2)
        perc_list.append(max_perc)

    np.savetxt(X=perc_list, fname=working_dir+"/percolation/original/"+nam+".txt", fmt="%.5f")


Computing percolation points for set: n


100%|██████████| 256/256 [02:47<00:00,  1.53it/s]


Computing percolation points for set: n1o


100%|██████████| 256/256 [04:36<00:00,  1.08s/it]


Computing percolation points for set: n2o


100%|██████████| 256/256 [06:54<00:00,  1.62s/it]


Computing percolation points for set: n3o


100%|██████████| 256/256 [07:50<00:00,  1.84s/it]


Computing percolation points for set: o


100%|██████████| 70/70 [02:34<00:00,  2.21s/it]


In [ ]:
#for randomizations

for k in range(0,1): # in this case done for the first randomized system, enlarge the number to get more
    if not os.path.isdir(working_dir+"/topology/percolation/"+str(k)):
        os.mkdir(working_dir+"/topology/percolation/"+str(k))
    for nam, lst in zip(names_lists, index_lists):
        print(nam)
        perc_list=[]
        for i in tqdm(range(len(lst))):
            net = mxp.VirusMultiplex_from_dirlist([working_dir+"/SynteticViruses/"+str(k)+"/"+a for a in virus_metadata.iloc[lst[i]]["virus"]])
            
            tensor=mxp.build.get_node_tensor_from_network_list(net.g_list)

            order = mxp.versatility.get_multi_RW_centrality_edge_colored(tensor)
            order = order.sort_values("vers")["phy nodes"].to_numpy()
            g_agg = mxp.build.get_aggregate_network(tensor)
    
            perc_agg_2 = gt.topology.vertex_percolation(g_agg, order, second=True)[0]
            max_perc = np.argmax(perc_agg_2)/len(perc_agg_2)
            perc_list.append(max_perc)
    
        np.savetxt(X=perc_list, fname=working_dir+"/topology/percolation/"+str(k)+"/"+nam+".txt", fmt="%.5f")
        

n


100%|█████████████████████████████████████████| 256/256 [02:33<00:00,  1.67it/s]


n1o


100%|█████████████████████████████████████████| 256/256 [04:14<00:00,  1.00it/s]


n2o


100%|█████████████████████████████████████████| 256/256 [05:36<00:00,  1.31s/it]


n3o


100%|█████████████████████████████████████████| 256/256 [04:00<00:00,  1.06it/s]


o


100%|███████████████████████████████████████████| 70/70 [02:33<00:00,  2.19s/it]


## Block structure

The mesoscale structure of the multilayer networks is further explored by comparing the results of the **Degree-corrected Stochastic Block Model (DCSBM)**. 

The 2 analyzed quantities are: 
- the number of non-empty modules 
- the modularity of the systems.

<img src="images/SBM.png" alt="sbm" width="400"/>

In [ ]:
### DCSBM block structure analysis ###
######################################

if not os.path.isdir(working_dir+"/modules/original"):
    os.mkdir(working_dir+"/modules/original")
if not os.path.isdir(working_dir+"/modularity/original"):
    os.mkdir(working_dir+"/modularity/original")
for nam, lst in zip(names_lists, index_lists):
    print("Analyzing block structure for set:", nam)
    mods_list=[]
    mody_list=[]
    for i in tqdm(range(len(lst))):
        net = mxp.VirusMultiplex_from_dirlist([working_dir+"/SyntheticViruses/original/"+a for a in virus_metadata.iloc[lst[i]]["virus"]])
        
        mod_res = mxp.mesoscale.get_mod(g_multi=net.g_multi, n_iter=1)
        mods_list.append(mod_res[0])
        mody_list.append(mod_res[1])

    np.savetxt(X=mods_list, fname=working_dir+"/modules/original/"+nam+".txt", fmt="%d")
    np.savetxt(X=mody_list, fname=working_dir+"/modularity/original/"+nam+".txt", fmt="%.5f")
        

Analyzing block structure for set: n


  0%|                                                     | 0/1 [00:00<?, ?it/s]

In [ ]:
#randomizations

for k in range(0,1): # in this case done for the first randomized system, enlarge the number to get more
    if not os.path.isdir(working_dir+"/modules/"+str(k)):
        os.mkdir(working_dir+"/modules/"+str(k))
    if not os.path.isdir(working_dir+"/modularity/"+str(k)):
        os.mkdir(working_dir+"/modularity/"+str(k))
    for nam, lst in zip(names_lists, index_lists):
        print(nam)
        mods_list=[]
        mody_list=[]
        for i in tqdm(range(len(lst))):
            net = mxp.VirusMultiplex_from_dirlist([working_dir+"/SyntheticViruses/"+str(k)+"/"+a for a in virus_metadata.iloc[lst[i]]["virus"]])
            
            mod_res = mxp.mesoscale.get_mod(g_multi=net.g_multi, n_iter=1)
            mods_list.append(mod_res[0])
            mody_list.append(mod_res[1])
    
        np.savetxt(X=mods_list, fname=working_dir+"/modules/"+str(k)+"/"+nam+".txt", fmt="%d")
        np.savetxt(X=mody_list, fname=working_dir+"/modularity/"+str(k)+"/"+nam+".txt", fmt="%.5f")

# Data for ML classifier

In [27]:
n_iters_ML = 3000
np.random.seed(100)

#N
n_virus_indexes_ML = np.array([np.random.choice(virus_nonco_idx, 4, replace=False) for i in range(n_iters_ML)])

#N1O
n1o_virus_indexes_ML = []
for i in range(n_iters_ML):
    onco_pick = np.random.choice(virus_onco_idx, 1, replace=False)
    nonco_pick = np.random.choice(virus_nonco_idx, 3, replace=False)
    n1o_virus_indexes_ML.append(np.concatenate([nonco_pick, onco_pick]))
n1o_virus_indexes_ML = np.array(n1o_virus_indexes_ML)

#np.savetxt(X=n_virus_indexes_ML, fname="data/MultilayerIndexes/n_ML.txt", fmt="%d")
#np.savetxt(X=n1o_virus_indexes_ML, fname="data/MultilayerIndexes/n1o_ML.txt", fmt="%d")

In [ ]:
##############################CENTR ONLY###########################################################
n_top = 2000
np.random.seed(100)

for nam, lst in zip(["n", "n1o"], [n_virus_indexes_ML, n1o_virus_indexes_ML]):
    centr_vec = []
    pos_vec = []
    for i in tqdm(range(n_iters_ML)):
        net = mxp.VirusMultiplex_from_dirlist([working_dir+"/SyntheticViruses/original/"+a for a in itemgetter(*lst[i])(virus_dict)])
        tensor = mxp.build.get_node_tensor_from_network_list(net.g_list)
        
        res_df = mxp.versatility.get_multi_RW_centrality_edge_colored(node_tensor=tensor, cval=0.15)

        list_res = np.array(list(net.node_map.keys()))[res_df.sort_values("vers", ascending=False).index[:50]]

        centr_norm = np.zeros(len(human_nodes))
        centr_norm[np.array(itemgetter(*list(net.node_map.keys()))(human_map))] = res_df["vers"].to_numpy()
        centr_norm=centr_norm/max(centr_norm)

        # get top 2000 nodes and save their normalized centrality values with position in the human PPI network
        top_nodes = np.argsort(centr_norm)[-n_top:]
        centr_vec.append(centr_norm[top_nodes])
        pos_vec.append(top_nodes)

    np.savetxt(X=centr_vec, fname=working_dir+"/ML_data/"+nam+".txt", fmt="%.4e")
    np.savetxt(X=pos_vec, fname=working_dir+"/ML_data/"+nam+"_pos.txt", fmt="%d")